**Grading the Graders**


Below is the code to clean and format the data with information on professors' respective pay and expenses from 2021 to 2023. The first step, done manually, was copying all the tables from https://finance.ubc.ca/reporting-planning-analysis/financial-reports under the Financial Information Act. Steps include Saving only pages with Professor financial information; copy and past all on a word doc, and this should yield information formatted like “last name, last name first name, number, number.” If it has multiple names consecutive after each other or more than two successive numbers, the formatting is incorrect. The rest of the code will not work. Fiscal year 2020 had this problem; thus, it was unusable. 

After completing the previous steps, we can go ahead and clean the data by running the following three chunks of code; from here, you will have a Word document with bullet points for every professor and their pay and expenses year over year. You can copy the entire document, remove the bullet points, and paste them into an Excel sheet. To clean the Excel, we select the column that all of our data is on and use the converted text to table or column tool to separate our data using the delimited option and separate using the indicator of a right curled bracket.  From here, we replace all the left brackets with empty space or nothing and all the double and triple spaces with single spaces.



In [1]:
import pandas as pd
from docx import Document
import re
import graphviz as gr

In [34]:

# Here we go from an unformatted word doc to the following format: '(Aamodt, Tor) (193,153) (5,597)'

doc_path = "C:/Users/Guill/OneDrive/Documents/2023.docx"

def combine_lines(doc_path):
    # Load the Word document
    doc = Document(doc_path)

    # Variable to keep track of the previous paragraph
    previous_para = None

    # Iterate through the paragraphs in the document
    for para in doc.paragraphs:
        # Check if the paragraph starts with a number
        if re.match(r'^\d', para.text.strip()):
            if previous_para:
                # Combine this paragraph with the previous one
                previous_para.text += ' ' + para.text
                # Clear the text in the current paragraph
                para.text = ''
        else:
            # Update the previous paragraph
            previous_para = para

    # Save the modified document
    file_path = "C:/Users/Guill/OneDrive/Documents/my_list23.docx"
    doc.save(file_path)

# Path to your Word document
combine_lines(doc_path)


['(Aamodt, Tor) (193,153) (5,597)', '(Abanto Salguero, Arleni Karina) (107,723) (393)', '(Abbassi, Arash) (109,136) (82)', '(Abdalkhani, Arman  -) (101,829) ()', '(Abdi, Ali) (238,203) (2,981)', '(Abdin, Yasmine) (121,196) (7,635)', '(Abdulai, Fatawu) (89,454) (8,049)', '(Abdul-Mageed, Muhammad) (105,795) (13,458)', '(Abe, Masumi) (99,796) (2,165)', '(Abedinifard, Mostafa) (124,090) (6,127)', '(Abel-Co, Karen  -) (134,722) ()', '(Abelló, Juan) (167,839) (8,755)', '(Abji, Tamiza  -) (79,276) ()', '(Ablona, Aidan) (86,901) (2,441)', '(Abolmaesumi, Purang) (261,609) (11,980)', '(Abraham, Ninan) (229,979) (12,334)', '(Abrutyn, Seth) (198,681) (12,886)', '(Abshire, Levonne) (126,395) (5,622)', '(Abu-Laban, Riyad) (115,717) (3,400)', '(Aburaed, Ahmed  -) (75,071) ()', '(Accili, Eric) (164,324) (602)', '(Accurso, Kathryn) (130,691) (3,771)', '(Acharya, Amrita) (108,052) (15)', '(Achermann, Reto) (94,736) (4,603)', '(Achugbue, Elsie  -) (127,404) ()', '(Ada, Kawa) (108,785) (574)', '(Adaji, If

In [ ]:
#This is more formatting, to account for other formatting problems encountered. 


# Load the uploaded Word document
doc_path = "C:/Users/Guill/OneDrive/Documents/my_list23.docx"
doc = Document(doc_path)

def reformat_document_ignore_page_numbers(doc):
    formatted_data = []
    previous_line = ""

    for para in doc.paragraphs:
        line = para.text.strip()

        # Skip empty lines and page numbers
        if not line or "Page |" in line:
            continue

        # Check if the line is a continuation of a name (last name ends with a comma)
        if ',' in line and not any(char.isdigit() for char in line):
            previous_line = line  # Save the line for concatenation with the next line
        else:
            if previous_line:
                line = previous_line + ' ' + line  # Concatenate name parts
                previous_line = ""

            # Extract remuneration and expenses using regex and format the line
            numbers = re.findall(r'\d[\d,]*', line)
            if numbers:
                name = re.sub(r'\d[\d,]*', '', line).strip().rstrip(',')
                remuneration = numbers[0] if len(numbers) > 0 else ''
                expenses = numbers[1] if len(numbers) > 1 else ''
                formatted_line = f"({name}) ({remuneration}) ({expenses})"
                formatted_data.append(formatted_line)
            else:
                formatted_data.append(f"({line})")

    return formatted_data

# Process the document
reformatted_data = reformat_document_ignore_page_numbers(doc)


print(reformatted_data)

In [35]:
# Here we put the rows as bullet points.

# Create a new Word document
document = Document()

# Create a list
my_list = reformatted_data

# Add the list to the Word document with each item on a new line (as paragraphs)
for item in my_list:
    document.add_paragraph(item, style='List Bullet')

# Save the Word document
file_path = "C:/Users/Guill/OneDrive/Documents/my_list23.docx"

# Save the Word document to the specified location
document.save(file_path)


In [4]:
# Load the new Excel file provided by the user
file_path = 'C:/Users/Guill/OneDrive/Documents/3years.xlsx'
data_3years = pd.read_excel(file_path)

# Display the first few rows of the dataset to understand its structure
data_3years.head()

# Aggregating wage data for each professor for each year to handle duplicates
aggregated_data = data_3years.groupby(['year', 'name']).agg({'wage': 'sum'}).reset_index()

# Pivoting the aggregated data to have years as columns for each professor
pivot_aggregated_data = aggregated_data.pivot(index='name', columns='year', values='wage')

# Calculating the percentage increase from 2021 to 2022 and from 2022 to 2023
pivot_aggregated_data['Increase_2021_to_2022'] = (pivot_aggregated_data[2022] - pivot_aggregated_data[2021]) / pivot_aggregated_data[2021] * 100
pivot_aggregated_data['Increase_2022_to_2023'] = (pivot_aggregated_data[2023] - pivot_aggregated_data[2022]) / pivot_aggregated_data[2022] * 100

# Computing average and median percentage increases
average_increase_2021_to_2022 = pivot_aggregated_data['Increase_2021_to_2022'].mean()
median_increase_2021_to_2022 = pivot_aggregated_data['Increase_2021_to_2022'].median()
average_increase_2022_to_2023 = pivot_aggregated_data['Increase_2022_to_2023'].mean()
median_increase_2022_to_2023 = pivot_aggregated_data['Increase_2022_to_2023'].median()

# Identifying outliers - using 1.5 times the IQR
iqr_2021_to_2022 = pivot_aggregated_data['Increase_2021_to_2022'].quantile(0.75) - pivot_aggregated_data['Increase_2021_to_2022'].quantile(0.25)
outlier_threshold_2021_to_2022 = pivot_aggregated_data['Increase_2021_to_2022'].quantile(0.75) + 1.5 * iqr_2021_to_2022

iqr_2022_to_2023 = pivot_aggregated_data['Increase_2022_to_2023'].quantile(0.75) - pivot_aggregated_data['Increase_2022_to_2023'].quantile(0.25)
outlier_threshold_2022_to_2023 = pivot_aggregated_data['Increase_2022_to_2023'].quantile(0.75) + 1.5 * iqr_2022_to_2023


print(f"The average pay increase between 2021 and 2022 is {average_increase_2021_to_2022}%")
print(f"The median pay increase between 2021 and 2022 is {median_increase_2021_to_2022}%")
print(f"The average pay increase between 2022 and 2023 is {average_increase_2022_to_2023}%")
print(f"The median pay increase between 2022 and 2023 is {median_increase_2022_to_2023}%")
print(f"A statistical outlier for 2021 to 2022 is above {outlier_threshold_2021_to_2022}%")
print(f"A statistical outlier for 2022 to 2023 is above {outlier_threshold_2022_to_2023}%")

The average pay increase between 2021 and 2022 is 4.549118095266303%
The median pay increase between 2021 and 2022 is 3.7152863000755936%
The average pay increase between 2022 and 2023 is 3.9439416027081253%
The median pay increase between 2022 and 2023 is 2.9729566898584894%
A statistical outlier for 2021 to 2022 is above 13.62753137578172%
A statistical outlier for 2022 to 2023 is above 12.566120717902018%


In [14]:
# Here I have to take out professors with duplicate names (2 differnt people with the same name, this only removes 26 professors out of about 20,000). We do this since we cant tell them apart. 

# Load the new Excel file provided by the user
file_path = 'C:/Users/Guill/OneDrive/Documents/professors_data.csv'
data_3years = pd.read_csv(file_path, encoding='latin1')
# Dropping all duplicate names for the same year from the original data
data_no_duplicates = data_3years.drop_duplicates(subset=['Year', 'Professor'], keep=False)

# Displaying the first few rows of the dataset after removing duplicates
data_no_duplicates.head()
data_no_duplicates.to_csv("C:/Users/Guill/OneDrive/Documents/prof_data_noD.csv")


In [53]:
# Here I add the dummy to professors considered outliers (about 13% for both 21-22 and 22-23)

# Read the DataFrame from the CSV file
data = pd.read_csv("C:/Users/Guill/OneDrive/Documents/firstnameprofwdumy.csv")

# Update the function to correctly mark the end of contracts in the previous year
def did_contract_end_updated(professor_data):
    # Sort by year
    professor_data = professor_data.sort_values(by='Year')

    # Calculate the wage increase percentage year over year
    professor_data['WageIncrease'] = professor_data['Wage'].pct_change() * 100

    # Determine if the contract ended (more than 10% increase in the next year)
    professor_data['ContractEnded'] = (professor_data['WageIncrease'].shift(-1) > 13).astype(int)

    # If a professor is not present in the subsequent year, mark the contract as ended
    professor_data['ContractEnded'] = professor_data['ContractEnded'] | ((professor_data['Year'].shift(-1) - professor_data['Year'] > 1).astype(int))

    # Ensure the last year (2023) is marked as 0 (ongoing contract)
    if professor_data['Year'].iloc[-1] == 2023:
        professor_data['ContractEnded'].iloc[-1] = 0

    return professor_data

# Apply the updated function to each professor
updated_contract_endings = data.groupby('Professor').apply(did_contract_end_updated).reset_index(drop=True)

# Check the modified dataset
updated_contract_endings.head()


updated_contract_endings.to_csv('profdum.csv', index=False)  # Saves the file as 'updated_dataset.csv'


C:\Users\Guill\AppData\Local\Temp\ipykernel_18288\2070370067.py:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  updated_contract_endings = data.groupby('Professor').apply(did_contract_end_updated).reset_index(drop=True)


In [5]:
# Here i clean the data further so we can directly compare with the grades data and match professors, so Which is formatted "first name last name" while we have "last name, first name"
df = pd.read_csv("C:/Users/Guill/OneDrive/Documents/profwdumy.csv")
# Function to reformat the name from 'last, first' to 'first last'
def reformat_name(name):
    parts = name.split(", ")
    if len(parts) == 2:  # Ensure the name has two parts
        return parts[1] + " " + parts[0]
    else:
        return name  # Return the name as is if it doesn't have two parts

# Apply the function to the entire 'Professor' column
df['Professor'] = df['Professor'].apply(reformat_name)
df.head()
df.to_csv("C:/Users/Guill/OneDrive/Documents/firstnameprofwdumy.csv")



Below is the code for cleaning and preparing the grades data frame. This was much easier as the data is readily available from ubcgrades.ca. Here, I downloaded all the grades and combined them into one CSV. Since some classes have multiple professors who teach the same class, we split them up, keeping the same class information for the professors who taught the same class although only having one name under the professor column. 

In [7]:
df = pd.read_csv("C:/Users/Guill/OneDrive/Documents/grades19-23.csv")

# Remove rows where the 'Professor' column is empty
df_cleaned = df.dropna(subset=['Professor'])


In [9]:
# Function to split the 'Professor' column by ';'
def split_professors(row):
    # Split the 'Professor' field if it's not NaN
    if pd.notna(row['Professor']):
        professors = row['Professor'].split(';')
        # Create a new row for each professor and keep the other information the same
        return [row[:3].tolist() + [prof.strip()] + row[4:].tolist() for prof in professors]
    return []

# Apply the function to each row and create a list of lists
split_rows = [new_row for index, row in df_cleaned.iterrows() for new_row in split_professors(row)]

# Create a new dataframe from the list of lists
df_expanded = pd.DataFrame(split_rows, columns=df_cleaned.columns)




,Year,Course,Title,Professor,Enrolled,Avg,Std dev,High,Low,Fails,Passes,Fail Percentage
0,2019,ACAM390,Asian Migrations in a Global Context,Henry Yu,18,86.722222,3.177227,91,81,0,18,0.0
1,2019,ADHE327,Teaching Adults,Dave Smulders,20,83.000000,8.466653,97,67,0,20,0.0
2,2019,ADHE327,Teaching Adults,Carolina Palacios,15,82.866667,5.642526,92,71,0,15,0.0
3,2019,ADHE327,Teaching Adults,Gabriella Maestrini,15,82.866667,5.642526,92,71,0,15,0.0
4,2019,ADHE327,Teaching Adults,Dave Smulders,22,85.772727,8.439882,96,68,0,22,0.0


This last chunk of code merges both data sets since combining all the data sets was quite a bit more computationally intensive. I broke them down year by year so we don't have to account for the correct years. The 2021 fiscal year accounts for the 2020 academic year, so I split the data sets based on their year and only merged the right fiscal and academic years. From there, I merged them back and deleted rows with no matches. 

In [78]:
professors_df = pd.read_csv('C:/Users/Guill/OneDrive/Documents/profwdum23.csv')
grades_df = pd.read_csv('C:/Users/Guill/OneDrive/Documents/grades22.csv')

merged_data21 = pd.merge(grades_df, professors_df, on='Professor', how='left')

merged_data21.to_csv("C:/Users/Guill/OneDrive/Documents/Alltogether23.csv")


In [79]:

# Load the dataframes from CSV files
df1 = pd.read_csv('C:/Users/Guill/OneDrive/Documents/Alltogether21.csv')
df2 = pd.read_csv('C:/Users/Guill/OneDrive/Documents/Alltogether22.csv')
df3 = pd.read_csv('C:/Users/Guill/OneDrive/Documents/Alltogether23.csv')
# If you have more files, load them as well

# Combine the dataframes by stacking them below each other
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# If there are any specific columns you want to check for missing values and drop those rows, you can do that here
columns_to_check = [
    'Unnamed: 0_y', 'Year_y', 'Wage', 'Cost', 
    'ContractEnded'
]
combined_df_cleaned = combined_df.dropna(subset=columns_to_check, how='all')

# Save the cleaned combined dataframe to a new CSV file
combined_df_cleaned.to_csv('C:/Users/Guill/OneDrive/Documents/Alltogetherall.csv', index=False)
